In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
import re

sns.set()

In [2]:
study_df0 = pd.read_csv("data/anticonflict_study_dataframe.csv",low_memory=False)
study_df = study_df0.copy()
study_df.shape

(24471, 500)

In [3]:
#coercing values which indicate errors or missingness to NaN
study_df = pd.DataFrame(study_df.apply(lambda x: np.where(x.isin([
    "-99              ", "-77              ","","-98              ","-97              ",
    "-55              ","-88              ","-66              ","[MASKED BY ICPSR]"]),np.nan,x)))

#dropping columns where over 50% of values are NaN, rows where all values are NaN,
#and rows where student IDs are invalid
#study_df.drop(columns=study_df.columns[(study_df.isna()).mean()>0.5].values, inplace=True)
study_df = study_df[~(study_df.isna()).all(1)]
study_df = study_df[(study_df["ID"] != "--blank--") & (study_df["ID"] != "999")]
study_df.dropna(subset=["ID","TREAT","SCHTREAT"],inplace=True)

study_df.shape

(22721, 500)

In [4]:
#filter dataframe based on relevant variables, selected for their
#importance in the causal graph
relevant_vars = pd.read_csv("data/Limited_VariableInclusion250_v2.csv",header=None)[0].unique()
relevant_vars3 = [col.translate({ord("("):"",ord(")"):""}).upper() for col in relevant_vars]

mtest_df = (study_df.copy())[relevant_vars3]
mtest_noms = [col for col in mtest_df.columns if ("ST" in col) & ("CN" in col)]
mtest_df.drop(columns=mtest_noms,inplace=True)
mtest_df.shape

In [6]:
#filter dataframe based on a list of variables that will factor
#into our "Final Anti-conflict Score" response variable
response_var_list = pd.read_csv("data/Response_VariableInclusion250.csv",header=None)[0]
response_var_names = [str(col).translate({ord("("):"",ord(")"):""}).upper() for col in response_var_list]
mtest_response = mtest_df[response_var_names]

In [7]:
#add variables that represent school records of students' conflicts;
#create a new variable to capture total instances of conflict
de_var_names = [col for col in mtest_response.columns if "DE" in col]
de_imputed = mtest_response.loc[:,de_var_names].apply(lambda x: np.where(x.isna(),0,x.fillna(0).astype(int)))
mtest_response = pd.concat([mtest_response.drop(columns=de_var_names),de_imputed],axis=1)
mtest_response.head(2)

,DN1W2,DN2W2,DN3W2,DN4W2,DN5W2,DN6W2,DN7W2,DN8W2,DN9W2,DN10W2,...,DE43,DE44,DE45,DE46,DE47,DE48,DE49,DE50,DE51,DE98
0,(3) 2-3 times/week,(4) Every day,(4) Every day,(3) 2-3 times/week,(2) About 1 time/week,(1) 1-2 times/month,(2) About 1 time/week,(3) 2-3 times/week,(3) 2-3 times/week,(3) 2-3 times/week,...,0,0,0,0,0,0,3,0,0,0
1,(3) 2-3 times/week,(4) Every day,(0) Never,(3) 2-3 times/week,(4) Every day,(3) 2-3 times/week,(1) 1-2 times/month,(1) 1-2 times/month,(3) 2-3 times/week,(1) 1-2 times/month,...,0,0,0,0,0,0,11,0,0,0


In [8]:
#create dummy variables for the response variables, since most are categorical
#include additional indicator variables for missing values
r_dummy_df = pd.get_dummies(mtest_response,drop_first=True,prefix_sep='_DUMMY',dummy_na=True)
r_dummy_df.columns = [((col.split()[0])).translate({ord("("):"",ord(")"):"",ord("["):"",ord("]"):""})\
                    for col in (r_dummy_df.columns)]

def retain_nans(x):
    """
    Takes a series produced by get_dummies(), uses the NaN columns
    such that missing values are preserved as missing and returns the series with preserved
    missingness. 
    """
    try:
        return np.where(r_dummy_df[x.name[:-1]+"nan"]==1,np.nan,x)
    except:
        #print("Except: "+x.name+" -- with "+x.name[:-1]+"nan")
        return x

#retain missingness in the response variables, drop dummy NaN columns
r_dummy_df = r_dummy_df.apply(lambda x: retain_nans(x))
r_dummy_df = r_dummy_df.loc[:,~r_dummy_df.columns.str.endswith("nan")]

r_dummy_df.isna().mean().sort_values().tail(5)

DNCL5W2_DUMMY1    0.419788
DNCL4W2_DUMMY1    0.419788
DNCL1W2_DUMMY1    0.419788
DNCL3W2_DUMMY1    0.419788
DNCL7W2_DUMMY1    0.419788
dtype: float64

In [9]:
#(Q 3.4 -- missing data imputation)
def fit_mice(dummy_df):
    """
    Given a dataframe of dummy variables with retained missingness,
    use the miceforest package's MICE algorithm to impute missing 
    values five separate times. Return an array of impited dataframes
    """
    
    results = []
    
    #creates a MICE imputation kernel based on the dummy dataframe
    #plans to create five seperate dataframes based on different
    #randomly produced results by the random forests underlying the
    #algorithm. See the package documenation for further information:
    #https://miceforest.readthedocs.io/en/latest/
    kernel = mf.ImputationKernel(
      dummy_df,
      datasets=5,
      save_all_iterations=True,
      random_state=396
    )
    
    #run the imputaton algorithm five times
    kernel.mice(5)

    for k in range(5):
        results.append(kernel.complete_data(k))
        
    return results

In [11]:
r_dummy_df.to_csv("data/full_unimputed_response_df.csv")

In [13]:
#get relevant explanatory variables, selecting both individually
#identified variables and variables from the first survey
exo_mtest_df = pd.concat([mtest_df.loc[:,relevant_vars3[5:12]],
                          study_df[[col.replace("W2","") for col in response_var_names\
                                    if (("W2" in col)&(col.replace("W2","") in study_df.columns))]]],axis=1)

GENDER      0.000880
GR          0.000880
AGEC_NEW    0.012411
COLL        0.026055
PN1         0.029576
              ...   
DNCL3       0.291492
DNCL1       0.291492
DNCL6       0.291492
DNCL5       0.291492
DNCL7       0.291492
Length: 72, dtype: float64

In [14]:
#get dummy variables and run retain_nans(x) to preserve 
#column missingness
e_dummy_df = pd.get_dummies(exo_mtest_df,drop_first=True,dummy_na=True)
e_dummy_df = e_dummy_df.apply(lambda x: retain_nans(x))
e_dummy_df = e_dummy_df.loc[:,~e_dummy_df.columns.str.endswith("nan")]

#run the MICE random forest algorithm
e_imputed_df = fit_mice(e_dummy_df)

#save the imputed dataframes
for k in range(5):
    e_imputed_df[k].to_csv(("data/full_imputed_predictor_df_iter"+str(k)+".csv"))

In [15]:
#identifies treatment variables and index variables in the dataset
roots_vars = [col for col in study_df if \
              (("RTSM" in col)|(col=="ID")|(col=="SCHID")|(col=="TREAT")|\
              (col=="SCHTREAT")|(col=="UID"))]

#filters based on treatment variables, coerces series to 
#appropriate data types
study_df_trt = study_df.loc[:,roots_vars]
study_df_trt["ID"] = study_df_trt["ID"].astype(int)
study_df_trt["SCHID"] = study_df_trt["SCHID"].astype(int)
study_df_trt["UID"] = study_df_trt["UID"].astype(int)

#get dummies and save file -- no need for data imputation
study_df_trt = pd.get_dummies(study_df_trt,drop_first=True)
study_df_trt.to_csv("data/full_unimputed_trt_df.csv")

In [16]:
#checkpoint for saving dataframe and reloading it
response_vars_df1 = pd.read_csv("data/full_unimputed_response_df.csv").iloc[:,1:]
response_vars_df1.shape

(22721, 222)

In [17]:
#a second checkpoint
predictor_vars_df1 = (pd.read_csv("data/full_imputed_predictor_df_iter0.csv")).iloc[:,1:]
predictor_vars_df1.shape

(22721, 157)

In [18]:
def split_cnum(cnum_var):
    """
    Given a string, return the values after initial numeric values
    """
    cnum_var = cnum_var[1:]
    nums = cnum_var.rstrip('0123456789')
    var_name = cnum_var[len(nums):]
    return var_name

#(Q 3.2 -- Baseline Anti-Conflic Score)
def create_composite_vars(full_df,predictors=True):
    """
    Construct the Baseline and Final Anti-Conflict scores. 
    """
    
    #determine variables to include based on whether they are predictors
    #manipulate variable names and string variables to ensure that predictor
    #and response variables have aligned syntax
    if predictors: 
        trs = ""
        colname = "Baseline_Conflict_Score"
        full_df.columns = [col.replace("(","DUMMY").split(")")[0] for col in full_df.columns]
    else:
        trs = "W2"
        colname="Composite_Conflict_Score"
    
    #get total number of disciplinary records for a student
    de_var_names = [col for col in full_df.columns if "DE" in col]
    full_df["DE_sum"] = full_df.loc[:,de_var_names].sum(axis=1)
    
    #create lists of variables based on the number of possible responses
    #and whether high positive values (e.g. TRUE, Often, Always) contribute
    #positively or negatively to the anti-conflict scores
    binary_positive_vars = [col for col in [(str(col).upper())+"_DUMMY1" for col in ("dncl10w2,dncl11w2,dncl12w2,pncl1w2,pncl2w2,"+\
                            "pncl7w2,pncl12w2,pncl13w2,cscaw2,clhcw2,flihcw2,tomenw2,"+\
                            "tomepgw2,tomelnw2,tomesuw2,tomehdw2,dncl9w2,dncl10w2,dncl11w2,"+\
                            "dncl12w2,cbnpw2".replace("w2",trs)).split(",")] if col in full_df.columns]
    binary_negative_vars = [col for col in [(str(col).upper())+"_DUMMY1" for col in ("dncl1w2,dncl2w2,dncl3w2,dncl4w2,dncl5w2,dncl6w2,"+\
                            "dncl7w2,dncl8w2,dncl14w2,pncl3w2,pncl4w2,pncl5w2,pncl6w2,cilw2,cflw2,"+\
                            "cmosw2,cbiw2,cgiw2,tomeew2,tomempw2,tomergw2,"+\
                            "tomerbw2,tomemfw2,tomepmw2,tomethpw2,tomerew2,tomesgw2"\
                                                                     .replace("w2",trs)).split(",")] if col in full_df.columns]
    cat3_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("infdw2,infcw2".replace("w2",trs))\
                                                 .split(",")] for i in range(1,3)] if col in full_df.columns]
    cat4_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("dn9w2,dn10w2,dn11w2,dn12w2".replace("w2",trs))\
                                                 .split(",")] for i in range(1,4)] if col in full_df.columns]
    cat4_negative_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("dn14w2,dn1w2,dn2w2,dn3w2,dn4w2,dn5w2,dn6w2,dn7w2,dn8w2".replace("w2",trs))\
                                                 .split(",")]\
                           for i in range(1,4)] if col in full_df.columns]
    cat5_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("pn1w2,pn2w2,pn10w2,pn11w2,pn12w2,pn13w2".replace("w2",trs)).split(",")]\
                          for i in range(1,5)] if col in full_df.columns]
    cat5_negative_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("pn3w2,pn4w2,pn5w2,pn6w2,pn7w2,pn9w2".replace("w2",trs)).split(",")]\
                           for i in range(1,5)] if col in full_df.columns]
    #cat3_negative_vars: none
    
    #aggregate variables by the aforementioned variable lists
    full_df["binary_vars"] = full_df.loc[:,binary_positive_vars]\
    .sum(axis=1) - full_df.loc[:,binary_negative_vars].sum(axis=1)

    #combine survey results with disciplinary records to produce
    #a final anti-conflict score
    comp_var = full_df["binary_vars"] - \
    (full_df["DE_sum"] * 0.25)

    return comp_var

In [19]:
#create the Baseline_Conflict_Score (aka Baseline Anti-conflict Score)
predictor_comp_df1 = predictor_vars_df1.copy()
predictor_comp_df1["Baseline_Conflict_Score"] = create_composite_vars(predictor_vars_df1,predictors=True)

#create the Composite_Conflict_Score (AKA final anti-conflict score)
response_comp_df1 = response_vars_df1.copy()
response_comp_df1["Composite_Conflict_Score"] = create_composite_vars(response_vars_df1,predictors=False)

#get treatment variables
trt_comp_df1 = pd.read_csv("data/full_unimputed_trt_df.csv")

#concatenate covariate/confounder, treatment, and response variables into one dataframe
complete_vars_df1 = pd.concat([trt_comp_df1,response_vars_df1, response_comp_df1,
                               predictor_vars_df1, predictor_comp_df1],axis=1)

complete_vars_df1.to_csv("data/preprocessed_conflict_data_iter0MICE.csv")

Index(['AGEC_NEW', 'GPA', 'GENDER_(1) Boy', 'GR_(1) 6th grade',
       'GR_(2) 7th grade', 'GR_(3) 8th grade', 'COLL_(1) Yes',
       'COLL_(2) Maybe', 'COLL_(3) Don't know',
       'CELL_(1) Cell phone without Internet',
       ...
       'TOMEMP_(1) Yes', 'TOMELN_(1) Yes', 'TOMERG_(1) Yes', 'TOMERB_(1) Yes',
       'TOMEMF_(1) Yes', 'TOMEPM_(1) Yes', 'TOMETHP_(1) Yes', 'TOMERE_(1) Yes',
       'TOMESG_(1) Yes', 'TOMESU_(1) Yes'],
      dtype='object', length=157)
Index(['DE1', 'DE2', 'DE3', 'DE4', 'DE5', 'DE6', 'DE7', 'DE8', 'DE9', 'DE10',
       ...
       'TOMELNW2_DUMMY1', 'TOMERGW2_DUMMY1', 'TOMERBW2_DUMMY1',
       'TOMEMFW2_DUMMY1', 'TOMEPMW2_DUMMY1', 'TOMETHPW2_DUMMY1',
       'TOMEREW2_DUMMY1', 'TOMESGW2_DUMMY1', 'TOMESUW2_DUMMY1',
       'TOMEHDW2_DUMMY1'],
      dtype='object', length=222)
